In [204]:
#全局数据
IDlist = []
Agelist = []
Genderlist = []
Edulist = []
Datalist = []


#按行读取数据
file = open('/ml/sougou/user_tag_query.2W.TRAIN')
file_lines = file.readlines()
file_linenum = len(file_lines)
file.close()

#age:    0：未知年龄; 1：0-18岁; 2：19-23岁; 3：24-30岁; 4：31-40岁; 5：41-50岁; 6： 51-999岁
#gender: 0：未知1：男性2：女性
#edu:    0：未知学历; 1：博士; 2：硕士; 3：大学生; 4：高中; 5：初中; 6：小学

#通过正则表达式说去数据
import re
pattern = '([0-9A-Z]*)\s([0-6]{1})\s([0-2]{1})\s([0-6]{1})\s(.*)'

IDlist = []
Agelist = []
Genderlist = []
Edulist = []
Datalist = []

for i in range( 0 , file_linenum ):
    m = re.search( pattern , file_lines[i] )
    if m == None:
        print( '[ERROR]bad lines:' , i )
        break
    else:
        IDlist.append(m.group(1))
        Agelist.append(m.group(2))
        Genderlist.append(m.group(3))
        Edulist.append(m.group(4))
        Datalist.append(m.group(5))

print( "读取原始数据%d条" % (len(IDlist)) )


#文本处理
import io
import os
import jieba.posseg as pseg

stopwords = [line.strip().decode('utf-8', 'ignore') for line in open('/ml/sougou/stopword1').readlines()]
print( "读取停用词%d条" % (len(stopwords)) )

def mycut( strCnText ):
    words = pseg.cut(strCnText)
    nword = []
    for w in words:
        #去除一些不关心的词:[c  连词][p  介词][q  量词][t  时间词][m  量词][eng  英文字母]
        if((w.flag != 'c' and w.flag != 'p' and w.flag != 'q' and w.flag != 't'and w.flag != 'm'and w.flag != 'eng') and len(w.word)>1):
            #通过停用词库去除一些不需要考虑的词汇
            if w.word not in stopwords:
                nword.append(w.word)
            
    return nword

#开始对数据进行读取
org_cut_data_file = '/ml/sougou/orgcut.dat'
def create_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == True ):
        cget = raw_input("File exist,cover it?(y / n)")
        if ( cget == 'y' ):
            os.remove( org_cut_data_file )
        else:
            return
        
    #创建文件
    print( "开始创建文件....." )
    fp = open( org_cut_data_file , 'w' )
    for i in range( 0 , nRealSize ):
    #for i in range( 0 , 10 ):
        if ( i % 100 == 0 ):
            print( "%d / %d" % ( i , nRealSize ) )
            fp.flush()
        fp.write( ','.join(mycut(Datalist[i])).encode('utf') + '\n' )
    fp.close()
        
#加载数据
def load_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == False ):
        create_org_cut_data()
    lines = [line.strip().decode('utf', 'ignore') for line in open(org_cut_data_file).readlines()]
    orgcutdata = []
    for line in lines:
        orgcutdata.append( line.split(',') )
    return orgcutdata

#对数据进行二次处理，一方面是将数据改为空格间隔，另一方面是可以对数据进行进一步的处理
def org_to_final_cut_data(org_cut_data):
    cutdata = []
    for words in org_cut_data:
        #这里可以对words进行进一步的调整
        '''
        wordslen = len( words )
        for i in  range(0,wordslen)[::-1]:
            if ( len( words[i] ) != 2 ):
                words.pop(i)
        '''     
        cutdata.append( ' '.join(words) )
        #cutdata.append( ' '.join(set(words)) )
        
    return cutdata

org_cut_data = load_org_cut_data()
final_cut_data = org_to_final_cut_data( org_cut_data )
print( "读取切分后数据%d条" % len(final_cut_data) )

#这里增加一个对性别位置数据的剔除工作
print ( "原有数据个数:%d" % (len(org_cut_data)) )
for i in range( 0 , len(org_cut_data) )[::-1]:
    if Genderlist[i] == '0':
        IDlist.pop(i)
        Genderlist.pop(i)
        final_cut_data.pop(i)
        org_cut_data.pop(i)

print ( "剔除未知之后数据个数:%d" % (len(org_cut_data)) )


读取原始数据20000条
读取停用词1893条
读取切分后数据20000条
原有数据个数:20000
剔除未知之后数据个数:19576


In [205]:
#切分数据

import scipy as sp  
import numpy as np  
from sklearn.datasets import load_files  
from sklearn.cross_validation import train_test_split  
from sklearn.feature_extraction.text import  TfidfVectorizer
  
doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Genderlist, test_size = 0.3)  
#doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(org_cut_data, Genderlist, test_size = 0.3)  
print( "train:%d , test:%d" % ( len(doc_terms_train) , len(doc_terms_test) ) )

train:13703 , test:5873


In [211]:
#转换
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



# 初始化TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000,ngram_range=(1, 1), max_df=0.5, norm='l1', use_idf=True)
x_train = vectorizer.fit_transform(doc_terms_train)
x_test = vectorizer.transform(doc_terms_test)

#print( x_train[0] )


有待考虑：TfidfVectorizer ？= CountVectorizer + TfidfTransformer
'''python
#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer=CountVectorizer()

#该类会统计每个词语的tf-idf权值
transformer=TfidfTransformer()

x_vtrain = vectorizer.fit_transform(doc_terms_train)
x_vtest = vectorizer.transform(doc_terms_test)

#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
x_train = transformer.fit_transform(x_vtrain)
x_test = transformer.transform(x_vtest)

#x_train = vectorizer.fit_transform(doc_terms_train)
#x_test = vectorizer.transform(doc_terms_test)

allwords = vectorizer.get_feature_names()
print( "当前所有词汇数量%d" % ( len(allwords) ) )
'''

In [212]:
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
"""
model = GradientBoostingClassifier(n_estimators=200)  
    model.fit(train_x, train_y)  
    return model  
"""
def fun_LogisticRegression( x_train , y_train , x_test , y_test ):
    from sklearn.linear_model.logistic import LogisticRegression
    #model = LogisticRegression()
    model = LogisticRegression(C=2,penalty='l2')
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_MultinomialNB( x_train , y_train , x_test , y_test ):
    from sklearn.naive_bayes import MultinomialNB  
    model = MultinomialNB(alpha=0.01)
    print( model )
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_BernoulliNB( x_train , y_train , x_test , y_test ):
    from sklearn.naive_bayes import BernoulliNB  
    model = BernoulliNB()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_KNeighborsClassifier( x_train , y_train , x_test , y_test ):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_DecisionTreeClassifier( x_train , y_train , x_test , y_test ):
    from sklearn.tree import DecisionTreeClassifier
    model = DecisionTreeClassifier()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    
def fun_SVC( x_train , y_train , x_test , y_test ):
    from sklearn.svm import SVC
    model = SVC()
    clf = model.fit(x_train , y_train )
    predictions = clf.predict( x_test )
    print( precision_score(y_test, predictions, average='micro')   )
    

In [213]:
fun_LogisticRegression( x_train , y_train , x_test , y_test )
fun_MultinomialNB( x_train , y_train , x_test , y_test )
fun_BernoulliNB( x_train , y_train , x_test , y_test )
#fun_SVC( x_train , y_train , x_test , y_test )
#fun_DecisionTreeClassifier( x_train , y_train , x_test , y_test )
#fun_KNeighborsClassifier( x_train , y_train , x_test , y_test )

#0.797377830751


0.694704580283
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
0.672058573131
0.812872467223


In [214]:
from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import BernoulliNB  
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1, 1), max_df=0.5, norm='l1', use_idf=True)),
    ('clf', BernoulliNB(alpha=0.01)),
])
parameters = {
    'vect__max_df': (0.25, 0.5, 0.75),
}


grid = GridSearchCV( pipeline , parameters, n_jobs=-1, verbose=1, scoring='accuracy', cv=4)
grid.fit(doc_terms_train, y_train)
#grid.fit(x_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
best_parameters = grid.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(' \t %s: %r' % (param_name, best_parameters[param_name]))

#predictions = grid_search.predict(X_test)


Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   43.6s finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l1', preprocessor=None, smooth_idf=True...vocabulary=None)), ('clf', BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': (0.25, 0.5, 0.75)},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)
0.807706341677
 	 vect__max_df: 0.5
